# Schema test

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from helper_functions import *
# AWS
import boto3
import awswrangler as wr

# load secret keys
db_host = os.environ.get('DB_HOST')
db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')
db_port = os.environ.get('DB_PORT')

In [2]:
# get data
data = get_data()
data.head()

loading file:
s3://dend-data/capstone/load-data/crime-weather-sample-100-09-18.csv


,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm
0,2009-08-21 00:00:00,1,Theft,2900-2999,almeda plaza,16E20,driveway,91.652,101.246,56,0.0,0.0
1,2012-09-20 07:00:00,1,Aggravated Assault,3700-3799,faulkner,14D20,apartment,69.944,69.638,64,0.0,0.0
2,2011-06-30 00:00:00,1,Burglary,8300-8399,long point,5F20,misc. business non-specific,92.228,93.722,39,0.0,0.0
3,2010-08-05 06:00:00,1,Aggravated Assault,3900-3999,corder,14D10,bus stop,82.688,91.418,83,0.0,0.0
4,2011-09-04 10:00:00,1,Auto Theft,3900-3999,sherwood,3B10,apartment,77.846,78.620,70,0.0,0.0


In [3]:
data.head()

,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm
0,2009-08-21 00:00:00,1,Theft,2900-2999,almeda plaza,16E20,driveway,91.652,101.246,56,0.0,0.0
1,2012-09-20 07:00:00,1,Aggravated Assault,3700-3799,faulkner,14D20,apartment,69.944,69.638,64,0.0,0.0
2,2011-06-30 00:00:00,1,Burglary,8300-8399,long point,5F20,misc. business non-specific,92.228,93.722,39,0.0,0.0
3,2010-08-05 06:00:00,1,Aggravated Assault,3900-3999,corder,14D10,bus stop,82.688,91.418,83,0.0,0.0
4,2011-09-04 10:00:00,1,Auto Theft,3900-3999,sherwood,3B10,apartment,77.846,78.620,70,0.0,0.0


In [4]:
# CREATE TABLE
# OFFENSE
offense_table = create_table_from_df(
    data, 'offense_type', 'offense_name', 'offense_pk')

offense_table.head()

creating tables...


,offense_pk,offense_name
0,1,Theft
1,2,Aggravated Assault
2,3,Burglary
3,4,Auto Theft
4,5,Robbery


In [5]:
data.head()

,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm
0,2009-08-21 00:00:00,1,Theft,2900-2999,almeda plaza,16E20,driveway,91.652,101.246,56,0.0,0.0
1,2012-09-20 07:00:00,1,Aggravated Assault,3700-3799,faulkner,14D20,apartment,69.944,69.638,64,0.0,0.0
2,2011-06-30 00:00:00,1,Burglary,8300-8399,long point,5F20,misc. business non-specific,92.228,93.722,39,0.0,0.0
3,2010-08-05 06:00:00,1,Aggravated Assault,3900-3999,corder,14D10,bus stop,82.688,91.418,83,0.0,0.0
4,2011-09-04 10:00:00,1,Auto Theft,3900-3999,sherwood,3B10,apartment,77.846,78.620,70,0.0,0.0


In [6]:
offense_table

,offense_pk,offense_name
0,1,Theft
1,2,Aggravated Assault
2,3,Burglary
3,4,Auto Theft
4,5,Robbery
5,6,Rape


In [7]:
data.head()

,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm
0,2009-08-21 00:00:00,1,Theft,2900-2999,almeda plaza,16E20,driveway,91.652,101.246,56,0.0,0.0
1,2012-09-20 07:00:00,1,Aggravated Assault,3700-3799,faulkner,14D20,apartment,69.944,69.638,64,0.0,0.0
2,2011-06-30 00:00:00,1,Burglary,8300-8399,long point,5F20,misc. business non-specific,92.228,93.722,39,0.0,0.0
3,2010-08-05 06:00:00,1,Aggravated Assault,3900-3999,corder,14D10,bus stop,82.688,91.418,83,0.0,0.0
4,2011-09-04 10:00:00,1,Auto Theft,3900-3999,sherwood,3B10,apartment,77.846,78.620,70,0.0,0.0


In [8]:
data.offense_type.value_counts()

Theft                 57
Burglary              14
Auto Theft            11
Aggravated Assault    10
Robbery                7
Rape                   1
Name: offense_type, dtype: int64

In [10]:
new = pd.merge_ordered(left=data,right=offense_table,left_on='offense_type',right_on='offense_name')
new

,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,offense_name
0,2012-09-20 07:00:00,1,Aggravated Assault,3700-3799,faulkner,14D20,apartment,69.944,69.638,64,0.00,0.0,2,Aggravated Assault
1,2010-08-05 06:00:00,1,Aggravated Assault,3900-3999,corder,14D10,bus stop,82.688,91.418,83,0.00,0.0,2,Aggravated Assault
2,2012-02-27 22:00:00,1,Aggravated Assault,8300-8399,mcavoy,15E10,apartment,69.440,69.332,69,0.00,0.0,2,Aggravated Assault
3,2010-06-09 20:00:00,1,Aggravated Assault,2100-2199,briley,10H50,residence house,89.942,100.112,61,0.00,0.0,2,Aggravated Assault
4,2014-09-17 15:00:00,1,Aggravated Assault,6800-6899,hornwood,17E10,hotel motel inn etc.,83.408,92.048,79,0.38,0.0,2,Aggravated Assault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2015-09-23 22:00:00,1,Theft,10000-10099,westheimer,20G10,commercial parking lot or garage,91.454,88.358,24,0.00,0.0,1,Theft
96,2012-01-30 21:00:00,1,Theft,1300-1399,greens,6B60,apartment parking lot,68.252,67.640,61,0.00,0.0,1,Theft
97,2014-12-20 11:00:00,1,Theft,2400-2499,loop,18F20,hotel or motel parking lot,50.846,50.000,93,0.00,0.0,1,Theft
98,2017-03-14 17:00:00,1,Theft,6600-6699,indian falls,16E40,residence or house,58.100,55.436,39,0.00,0.0,1,Theft


In [11]:
tt = data.merge(offense_table, left_on='offense_type', right_on='offense_name',sort=False)
tt.head()

,date_time,offenses,offense_type,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,offense_name
0,2009-08-21 00:00:00,1,Theft,2900-2999,almeda plaza,16E20,driveway,91.652,101.246,56,0.0,0.0,1,Theft
1,2014-01-13 19:00:00,1,Theft,5800-5899,ranchester,20G10,road street sidewalk,69.494,67.874,37,0.0,0.0,1,Theft
2,2010-02-06 12:00:00,1,Theft,7300-7399,fm 1960,5F40,department discount store,44.384,41.846,88,0.0,0.0,1,Theft
3,2016-01-30 18:00:00,1,Theft,7800-7899,katy,5F10,other parking lot,72.194,71.654,54,0.0,0.0,1,Theft
4,2014-12-13 13:00:00,1,Theft,5900-5999,katy,2A50,other parking lot,60.998,61.214,94,0.0,0.0,1,Theft


In [12]:
tt.offense_name.value_counts()

Theft                 57
Burglary              14
Auto Theft            11
Aggravated Assault    10
Robbery                7
Rape                   1
Name: offense_name, dtype: int64

In [13]:
# #data = drop_add(data, offense_table, 'offense_type', 'offense_name')
# data = data.merge(offense_table, right_on="offense_type",left_on="offense_name", suffixes=(False, False))
# data.head()

In [14]:
data = drop_add_pk(data, offense_table, 'offense_type', 'offense_name')
data.head()

,date_time,offenses,block_range,street_name,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk
0,2009-08-21 00:00:00,1,2900-2999,almeda plaza,16E20,driveway,91.652,101.246,56,0.0,0.0,1
1,2014-01-13 19:00:00,1,5800-5899,ranchester,20G10,road street sidewalk,69.494,67.874,37,0.0,0.0,1
2,2010-02-06 12:00:00,1,7300-7399,fm 1960,5F40,department discount store,44.384,41.846,88,0.0,0.0,1
3,2016-01-30 18:00:00,1,7800-7899,katy,5F10,other parking lot,72.194,71.654,54,0.0,0.0,1
4,2014-12-13 13:00:00,1,5900-5999,katy,2A50,other parking lot,60.998,61.214,94,0.0,0.0,1


In [ ]:
data.offense_pk.value_counts()

In [ ]:
data.info()

In [15]:
# upload table
upload_table(offense_table, offense_table_insert, 'offense_table')

uploading: offense_table


In [ ]:
offense_table

In [ ]:
data.head()

In [16]:
# POLICE BEAT
police_beat_table = create_table_from_df(
    data, 'beat', 'beat_name', 'beat_pk')
data = drop_add_pk(data, police_beat_table, 'beat', 'beat_name')
# upload table
upload_table(police_beat_table, police_beat_table_insert,
             'police_beat_table')

creating tables...
uploading: police_beat_table


In [17]:
data.head()

,date_time,offenses,block_range,street_name,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk
0,2009-08-21 00:00:00,1,2900-2999,almeda plaza,driveway,91.652,101.246,56,0.00,0.0,1,1
1,2017-01-12 14:00:00,1,4600-4699,fuqua,service or gas station,66.560,67.514,98,0.00,0.0,4,1
2,2014-01-13 19:00:00,1,5800-5899,ranchester,road street sidewalk,69.494,67.874,37,0.00,0.0,1,2
3,2018-02-20 01:00:00,1,3700-3799,gessner,convenience store,72.680,73.832,89,0.55,0.0,1,2
4,2015-09-23 22:00:00,1,10000-10099,westheimer,commercial parking lot or garage,91.454,88.358,24,0.00,0.0,1,2


In [18]:
# PREMISE
premise_table = create_table_from_df(
    data, 'premise_description', 'premise_name', 'premise_pk')
data = drop_add_pk(data, premise_table,
                   'premise_description', 'premise_name')
# upload table
upload_table(premise_table, premise_table_insert, 'premise_table')

creating tables...
uploading: premise_table


In [19]:
data.head()

,date_time,offenses,block_range,street_name,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk
0,2009-08-21 00:00:00,1,2900-2999,almeda plaza,91.652,101.246,56,0.00,0.0,1,1,1
1,2010-08-27 01:00:00,1,9600-9699,chatfield,89.996,90.032,38,0.00,0.0,1,25,1
2,2009-12-14 05:00:00,1,7400-7499,azalea,52.808,52.340,97,0.00,0.0,1,36,1
3,2016-08-06 20:00:00,1,7100-7199,gracia,98.438,102.524,35,0.00,0.0,4,53,1
4,2010-02-01 16:00:00,1,3600-3699,penwood,42.746,39.146,84,0.76,0.0,4,56,1


In [20]:
data.offenses.value_counts()

1    99
2     1
Name: offenses, dtype: int64

In [21]:
premise_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   premise_pk    41 non-null     int64 
 1   premise_name  41 non-null     object
dtypes: int64(1), object(1)
memory usage: 784.0+ bytes


In [22]:
data.premise_pk.value_counts()

20    15
7      9
12     9
1      5
26     5
3      4
9      4
4      3
27     3
17     2
25     2
24     2
23     2
2      2
18     2
15     2
37     2
11     2
10     2
31     2
35     1
30     1
34     1
36     1
33     1
38     1
39     1
40     1
32     1
21     1
29     1
28     1
22     1
19     1
16     1
14     1
13     1
8      1
6      1
5      1
41     1
Name: premise_pk, dtype: int64

In [23]:
# ADDRESS
data.block_range.replace('UNK', '10-100', inplace=True)
data.block_range.replace('1.1103e+006-1.1104e+006', '10-100', inplace=True)
data['og_address'] = block_range_split(data)
# drop street_name & block range
data.drop(['block_range', 'street_name'], axis=1, inplace=True)

address_table = create_table_from_df(
    data, 'og_address', 'full_address', 'address_pk')
data = drop_add_pk(data, address_table, 'og_address', 'full_address')
# upload table
upload_table(address_table, address_table_insert, 'address_table')

creating tables...
uploading: address_table


In [24]:
address_table.head()

,address_pk,full_address
0,1,"2950 almeda plaza Houston, TX"
1,2,"9650 chatfield Houston, TX"
2,3,"7450 azalea Houston, TX"
3,4,"7150 gracia Houston, TX"
4,5,"3650 penwood Houston, TX"


In [25]:
address_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address_pk    100 non-null    int64 
 1   full_address  100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [26]:
data.head()

,date_time,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk
0,2009-08-21 00:00:00,1,91.652,101.246,56,0.00,0.0,1,1,1,1
1,2010-08-27 01:00:00,1,89.996,90.032,38,0.00,0.0,1,25,1,2
2,2009-12-14 05:00:00,1,52.808,52.340,97,0.00,0.0,1,36,1,3
3,2016-08-06 20:00:00,1,98.438,102.524,35,0.00,0.0,4,53,1,4
4,2010-02-01 16:00:00,1,42.746,39.146,84,0.76,0.0,4,56,1,5


In [27]:
data.address_pk.value_counts()

1      1
64     1
74     1
73     1
72     1
      ..
31     1
30     1
29     1
28     1
100    1
Name: address_pk, Length: 100, dtype: int64

In [28]:
# DATETIME
dt_table = create_table_from_df(
    data, 'date_time', 'date_time', 'date_time_pk')

# inster time data records
t = pd.to_datetime(dt_table['date_time'])

time_data = (dt_table.date_time_pk, dt_table.date_time, t.dt.hour,
             t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('date_time_pk', 'date_time', 'hour',
                 'day', 'week', 'month', 'year', 'weekday')
date_time_table = pd.DataFrame.from_dict(
    dict(zip(column_labels, time_data)))

data = data.merge(date_time_table, left_on='date_time',
                  right_on='date_time', suffixes=(True, True))
data.drop(['date_time', 'hour', 'day', 'week', 'month',
          'year', 'weekday'], axis=1, inplace=True)
# upload table
upload_table(date_time_table, datetime_table_insert, 'date_time_table')

creating tables...
uploading: date_time_table


<ipython-input-28-da14116904cf>:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)


In [29]:
date_time_table

,date_time_pk,date_time,hour,day,week,month,year,weekday
0,1,2009-08-21 00:00:00,0,21,34,8,2009,4
1,2,2010-08-27 01:00:00,1,27,34,8,2010,4
2,3,2009-12-14 05:00:00,5,14,51,12,2009,0
3,4,2016-08-06 20:00:00,20,6,31,8,2016,5
4,5,2010-02-01 16:00:00,16,1,5,2,2010,0
...,...,...,...,...,...,...,...,...
95,96,2013-07-06 06:00:00,6,6,27,7,2013,5
96,97,2017-03-25 01:00:00,1,25,12,3,2017,5
97,98,2010-08-05 06:00:00,6,5,31,8,2010,3
98,99,2014-09-17 15:00:00,15,17,38,9,2014,2


In [31]:
data.head()

,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk,date_time_pk
0,1,91.652,101.246,56,0.00,0.0,1,1,1,1,1
1,1,89.996,90.032,38,0.00,0.0,1,25,1,2,2
2,1,52.808,52.340,97,0.00,0.0,1,36,1,3,3
3,1,98.438,102.524,35,0.00,0.0,4,53,1,4,4
4,1,42.746,39.146,84,0.76,0.0,4,56,1,5,5


In [32]:
# CRIME FACT
crime_fact = data.reset_index()
crime_fact.rename(columns={'index': 'pk'}, inplace=True)
crime_fact.pk = crime_fact.pk + 1

In [35]:
crime_fact.head(20)

,pk,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,offense_pk,beat_pk,premise_pk,address_pk,date_time_pk
0,1,1,91.652,101.246,56,0.00,0.0,1,1,1,1,1
1,2,1,89.996,90.032,38,0.00,0.0,1,25,1,2,2
2,3,1,52.808,52.340,97,0.00,0.0,1,36,1,3,3
3,4,1,98.438,102.524,35,0.00,0.0,4,53,1,4,4
4,5,1,42.746,39.146,84,0.76,0.0,4,56,1,5,5
5,6,1,66.560,67.514,98,0.00,0.0,4,1,2,6,6
6,7,1,43.592,38.426,74,0.00,0.0,5,58,2,7,7
7,8,1,69.494,67.874,37,0.00,0.0,1,2,3,8,8
8,9,1,72.554,71.726,47,0.00,0.0,1,10,3,9,9
9,10,1,80.294,80.672,46,0.00,0.0,1,26,3,10,10


In [34]:
crime_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pk              100 non-null    int64  
 1   offenses        100 non-null    int64  
 2   temp            100 non-null    float64
 3   feels_like      100 non-null    float64
 4   humidity_per    100 non-null    int64  
 5   rain_vol_1h_mm  100 non-null    float64
 6   snow_vol_1h_mm  100 non-null    float64
 7   offense_pk      100 non-null    int64  
 8   beat_pk         100 non-null    int64  
 9   premise_pk      100 non-null    int64  
 10  address_pk      100 non-null    int64  
 11  date_time_pk    100 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 9.5 KB


In [ ]:
# upload table
upload_table(crime_fact, crime_fact_table_insert, 'crime_fact_table')